[Tabluar Quickstart](https://auto.gluon.ai/stable/tutorials/tabular_prediction/tabular-quickstart.html) を参考にAutoGluonを使ってみる。

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [4]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,178478,Bachelors,13,Never-married,Tech-support,Own-child,White,Female,0,0,40,United-States,<=50K
1,23,State-gov,61743,5th-6th,3,Never-married,Transport-moving,Not-in-family,White,Male,0,0,35,United-States,<=50K
2,46,Private,376789,HS-grad,9,Never-married,Other-service,Not-in-family,White,Male,0,0,15,United-States,<=50K
3,55,?,200235,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,0,0,50,United-States,>50K
4,36,Private,224541,7th-8th,4,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,El-Salvador,<=50K


In [7]:
subsample_size = 500
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,500.00000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,39.65200,189322.638000,10.118000,1040.906000,118.054000,40.802000
std,13.52393,94845.719575,2.582521,5462.912364,456.158416,12.168111
min,17.00000,22201.000000,2.000000,0.000000,0.000000,1.000000
25%,29.00000,129695.000000,9.000000,0.000000,0.000000,40.000000
50%,38.00000,183068.500000,10.000000,0.000000,0.000000,40.000000
75%,49.00000,230417.750000,13.000000,0.000000,0.000000,45.000000
max,85.00000,723746.000000,16.000000,99999.000000,2415.000000,99.000000


In [8]:
label = 'class'
train_data[label]

5372      <=50K
26971     <=50K
31907     <=50K
3850      <=50K
4075       >50K
          ...  
5099      <=50K
10160     <=50K
3539      <=50K
393       <=50K
9247       >50K
Name: class, Length: 500, dtype: object

In [9]:
train_data[label].describe()

count        500
unique         2
top        <=50K
freq         365
Name: class, dtype: object

In [10]:
train_data[label].value_counts()

 <=50K    365
 >50K     135
Name: class, dtype: int64

In [16]:
save_path = 'auto-gluon-models'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "auto-gluon-models/"
AutoGluon Version:  0.3.1
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12801.76 MB
	Train Dat

In [12]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


In [13]:
y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)

Predictions:  
 0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object


In [14]:
y_pred.value_counts()

 <=50K    8278
 >50K     1491
Name: class, dtype: int64

In [15]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.8264919643771113
Evaluations on test data:
{
    "accuracy": 0.8264919643771113,
    "balanced_accuracy": 0.698874666805007,
    "mcc": 0.47051332309340116,
    "f1": 0.5550013126804936,
    "precision": 0.7089201877934272,
    "recall": 0.4559965487489215
}
